<a href="https://colab.research.google.com/github/openchung/chatgpt-financial-bot/blob/main/Gemini_Financial_LlamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip -q  install -q --upgrade  llama-index
!pip -q  install -q --upgrade  llama-index-llms-gemini llama-index-embeddings-gemini llama-index-embeddings-openai
!pip -q  install -q --upgrade  fpdf PyPDF2

In [7]:
import os
from google.colab import userdata
from llama_index.llms.gemini import Gemini
from llama_index.core.llms import ChatMessage


GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
response = Gemini(api_key=GOOGLE_API_KEY, model="models/gemini-1.5-pro-latest").complete("誰是王建民？")
print(response)

王建民是一位來自台灣的**前職業棒球投手**。他最為人所知的是在美國職棒大聯盟（MLB）效力紐約洋基隊期間的出色表現。

以下是關於王建民的一些重要資訊：

* **出生日期：** 1980年3月31日（43歲）
* **出生地：** 台灣台南市
* **投球和擊球：** 右投右打
* **大聯盟生涯：** 2005年－2011年、2013年、2016年

**生涯成就：**

* 兩屆美國聯盟勝投王（2006、2007）
* 一次美國聯盟防禦率第二名（2006）
* 入選美國聯盟明星賽（2008）

王建民以其**伸卡球**聞名，這種球路能有效地讓打者擊出滾地球。他在洋基隊的巔峰時期是球隊的王牌投手，幫助球隊連續多年打進季後賽。

雖然傷病影響了王建民後期的職業生涯，但他仍然是台灣最傑出的棒球運動員之一，也是許多台灣人心目中的英雄。



In [9]:
# 如果想要改用最新的 gemini 1.5 pro 或調整參數，可以像這樣設定：
response = Gemini(
    api_key=GOOGLE_API_KEY,
    model_name="models/gemini-1.5-pro-latest",
    temperature=0.6,
    max_tokens=1024
).complete("誰是王建民？")
print(response)

王建民是一位來自台灣的**前職業棒球投手**，他在美國職棒大聯盟（MLB）效力過紐約洋基隊、華盛頓國民隊、多倫多藍鳥隊和堪薩斯皇家隊。 

以下是關於王建民的一些重要資訊：

* **巔峰時期：** 他在2006年和2007年效力於紐約洋基隊時達到巔峰，連續兩年贏得19場勝投，並在2006年獲得美國聯盟勝投王。
* **球風：** 王建民以其優異的伸卡球聞名，這種球路能讓打者擊出大量的滾地球出局。
* **傷病問題：**  不幸的是，他的職業生涯飽受傷病困擾，特別是在2008年和2009年遭受了嚴重的腿傷，影響了他的表現。
* **國際賽事：** 王建民也代表中華台北隊參加過多項國際賽事，包括世界棒球經典賽和奧運資格賽。
* **影響力：** 他是台灣最著名的棒球運動員之一，在台灣掀起了一股「王建民熱潮」，也鼓舞了許多台灣年輕球員追求棒球夢想。

如果您想了解更多關於王建民的資訊，可以在網路上搜尋他的名字，您會找到許多關於他的文章、影片和數據。 



In [11]:
# Chat
messages = [
    ChatMessage(role="user", content="誰是王建民？"),
    ChatMessage(role="assistant", content="王建民是臺灣職業棒球運動員，擔任投手，在美國職棒大聯盟9個球季累計68勝。"),
    ChatMessage(role="user", content="他是哪裡人？"),
]
response = Gemini(api_key=GOOGLE_API_KEY, model="models/gemini-1.5-pro-latest").chat(messages)
print(response)

assistant: 王建民出生於臺灣台南市。 



In [12]:
response = Gemini(api_key=GOOGLE_API_KEY, model="models/gemini-1.5-pro-latest").stream_complete("誰是王建民？")
for r in response:
    print(r.text, end="")

王建民是一位來自台灣的**前職業棒球投手**。他最為人所知的是在美國職棒大聯盟（MLB）效力紐約洋基隊期間的出色表現。

以下是關於王建民的一些重要資訊：

* **出生日期：** 1980年3月31日（43歲）
* **出生地：** 台灣台南市
* **投球和擊球：** 右投右打
* **大聯盟生涯：** 2005年-2011年，2013年，2016年
* **效力球隊：** 紐約洋基隊（2005-2009），華盛頓國民隊（2010-2011），多倫多藍鳥隊（2013），辛辛那提紅人隊（2016）

**王建民的成就：**

* 兩屆美國聯盟勝投王（2006、2007）
* 一次美國聯盟防禦率第二名（2006）
* 入選美國聯盟明星隊（2008）

王建民以其**沉穩的個性**、**優異的伸卡球**以及在球場上的**穩定表現**而聞名。他被認為是台灣最成功的棒球選手之一，也是許多台灣年輕球員的偶像。

雖然王建民的職業生涯因傷病而縮短，但他對棒球的熱愛和奉獻精神，以及他為台灣棒球帶來的榮耀，將永遠被球迷銘記。


In [13]:
# Embedding
from llama_index.embeddings.gemini import GeminiEmbedding

# embed_model = GeminiEmbedding(api_key=GOOGLE_API_KEY, model_name="models/embedding-001")
# 或者想使用最新的模型
embed_model = GeminiEmbedding(api_key=GOOGLE_API_KEY, model_name="models/text-embedding-004")
embeddings = embed_model.get_text_embedding("『IT空間』是一個有關基礎 IT 技術教學與分享的部落格。")
print(embeddings)
# [0.042481236, 9.7126205e-05, -0.06401828, -0.03270765, 0.066297136, -0.019318093, ......

[-0.004665864, 0.015415744, -0.01526715, -0.009305942, 0.030632917, 0.017968828, 0.02260423, 0.015370391, -0.008837132, 0.031362757, -0.017493682, -0.0035537367, 0.073688164, 0.0051427945, -0.02071991, -0.03277609, 0.06612981, 0.038504183, -0.110764824, 0.037625164, 0.018808749, -0.012127584, 0.039743587, 0.024125483, -0.033379097, -0.015526891, -0.010734212, -0.0032057462, -0.024293132, -0.026425159, 0.03662502, 0.064026624, 0.012761714, -0.031288363, 0.024030738, 0.017130619, 0.004531663, 0.04750668, 0.031853944, -0.039940424, -0.076829664, 0.0178506, -0.040523537, 0.031406626, 0.0070500425, -0.016196983, -0.005618048, 0.05324932, -0.051921085, 0.04875574, 0.00897834, -0.01086312, -0.04611966, 0.014475231, -0.0133456085, -0.0008056989, -0.017877415, -0.0009257727, 0.016268795, -0.0030733966, -0.034767255, 0.011924507, -0.01692051, -0.036672946, 0.011264716, -0.017766794, -0.030960409, -0.006492864, -0.085536085, 0.04807166, 0.009708211, -0.04010681, -0.06498526, -0.0072357627, -0.012

In [46]:
# @title 完成docs資料夾下Embedding
import os.path
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
# from llama_index.embeddings.openai import OpenAIEmbedding


PERSIST_DIR = "./storage"
GEMINI_API = GOOGLE_API_KEY
# OPENAI_API = "<YOUR_OPENAI_API>"

# 更改全域配置
Settings.embed_model = GeminiEmbedding(model_name="models/text-embedding-004", api_key=GEMINI_API)
# Settings.embed_model = OpenAIEmbedding(api_key=OPENAI_API)
Settings.llm = Gemini(api_key=GEMINI_API, model="models/gemini-1.5-pro-latest")


if not os.path.exists(PERSIST_DIR):
    documents = SimpleDirectoryReader("docs").load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)




In [54]:
query_engine = index.as_query_engine(
    similarity_top_k=1,
)
response = query_engine.query("請用產業別去分類，並用income為營業收入（單位為台幣）及eps每股盈餘（單位為%）去計算哪個產業最賺錢。")
# response = query_engine.query("葬送的芙莉蓮有什麼角色？")
print(response)

無法計算哪個產業最賺錢，因為提供的文件中沒有產業別與收入和每股盈餘的對應關係。 



In [14]:
# @title 取得上市公司
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
from fpdf import FPDF
import csv

url = 'https://mopsfin.twse.com.tw/opendata/t187ap03_L.csv'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')


In [15]:
# @title 公開資訊觀測站
def get_data_df(url, stockNo, year, season):
    form_data = {
            'encodeURIComponent': 1
            , 'step': 1
            , 'firstin': 1
            , 'off': 1
            , 'keyword4': ''
            , 'code1': ''
            , 'TYPEK2': ''
            , 'checkbtn': ''
            , 'queryName': 'co_id'
            , 'inpuType': 'co_id'
            , 'TYPEK': 'all'
            , 'isnew': False
            , 'co_id': stockNo
            , 'year': year
            , 'season': season
    }
    r = requests.post(url,form_data)

    # Check the status code of the response and print the response text for debugging
    if r.status_code == 200:
        #print(r.text) # Print the response text to see what the server is returning
        tables = pd.read_html(r.text)
        if len(tables) > 1:
            html_df = tables[1].fillna("")
            return html_df
        else:
            print(f"Error: The website did not return the expected number of tables for company {stockNo}, year {year}, season {season}. Response text: {r.text}")
            return pd.DataFrame()  # Return an empty DataFrame if no tables are found
    else:
        print(f"Error: Failed to fetch data from the website. Status code: {r.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame if the request fails


In [ ]:
# @title 試驗公開資訊觀測站函式
tables=get_data_df("https://mops.twse.com.tw/mops/web/ajax_t164sb04", 1101, '113', '01')
print(tables.columns)

In [17]:
# @title 取得產業別
categories = {}
categories_list = []
category_url = 'https://mopsfin.twse.com.tw/opendata/t187ap05_L.csv'
response = requests.get(category_url)
categories_csv = response.content.decode('utf-8')
spamreader = csv.reader(categories_csv.splitlines(), delimiter=',', skipinitialspace=True)

for row in spamreader:
    company_code = row[2]
    category_name = row[4]
    categories[company_code] = category_name
    categories_list.append(category_name)

print(len(categories_list))
print(categories['1101'])



1018
水泥工業


In [ ]:
# @title 產生PDF
from fpdf import FPDF
from PyPDF2 import PdfMerger, PdfFileReader # Use PdfMerger instead of PdfFileMerger

def create_pdf(input_file):
    # Create a new FPDF object
    pdf = FPDF()

    # Open the text file and read its contents
    with open(input_file, 'r', encoding='utf-8') as f: # Specify UTF-8 encoding when opening the file
        text = f.read()

    # Add a new page to the PDF
    pdf.add_page()

    # Set the font and font size, use a font that supports Chinese characters
    # Replace '/path/to/ArialUnicode.ttf' with the actual path to your font file
    pdf.add_font('ArialUnicode', '', 'FreeSerif.ttf', uni=True)
    pdf.set_font('ArialUnicode', size=12)

    # Write the text to the PDF
    pdf.write(5, text)

    # Save the PDF - no need for 'encoding' argument here
    pdf.output('output.pdf')

In [22]:
# @title 取得上市公司EPS資訊(TXT)
def write_data_totxt_from_opendata(company_datas, year, season):
    with open('docs/combined_data.txt', 'w', newline='') as file:
      df_csv = pd.read_csv("https://mopsfin.twse.com.tw/opendata/t187ap14_L.csv")
      for index in range(1, len(company_datas)):
          if not df_csv.empty:
              print(company_datas[index][0])
              result = df_csv.query("公司代號=="+company_datas[index][0])["營業收入"]
              if not result.empty:
                  company_income = result.values[0]
                  print(company_income)
                  # ... rest of your code to process company_income ...
              else:
                  print(f"No matching data found for company {company_datas[index][0]}")

              # Handle potential empty results for '營業利益' and '基本每股盈餘(元)'
              result_grossloss = df_csv.query("公司代號=="+company_datas[index][0])["營業利益"]
              if not result_grossloss.empty:
                  company_grossloss = result_grossloss.values[0]
              else:
                  company_grossloss = "N/A" # Or any other placeholder for missing data

              result_eps = df_csv.query("公司代號=="+company_datas[index][0])["基本每股盈餘(元)"]
              if not result_eps.empty:
                  company_eps = result_eps.values[0]
              else:
                  company_eps = "N/A" # Or any other placeholder for missing data

              write_string = f"公司名稱是{company_datas[index][1]}，簡稱為{company_codes[index][2]}, 公司代號是{company_codes[index][0]}，公司類別是{company_codes[index][3]}, 此公司是屬於{company_datas[index][3]}的產業別公司，在{year}年{season}季的財務報表, 營業收入合計為{company_income}, 營業利益為{company_grossloss}, 基本每股盈餘為{company_eps}。\n"
              print(write_string)
              file.write(write_string)
          else:
              print(f"Skipping company {company_datas[index][0]} because the DataFrame is empty.")
    # # 載入TXT文檔
    # input_file = "combined_data.txt"

    # # 將 TXT 另存為 PDF 文件
    # create_pdf(input_file)

In [44]:
# @title 取得上市公司EPS資訊(CSV)
def write_data_tocsv_from_opendata(company_datas, year, season):
    company_bus = ["income", "grossloss", "eps", "year", "season"]
    columns = company_datas[0] + company_bus
    data = []
    with open('combined_data.txt', 'w', newline='') as file:
      df_csv = pd.read_csv("https://mopsfin.twse.com.tw/opendata/t187ap14_L.csv")
      for index in range(1, len(company_datas)):
          if not df_csv.empty:
              print(company_datas[index][0])
              result = df_csv.query("公司代號=="+company_datas[index][0])["營業收入"]
              if not result.empty:
                  company_income = result.values[0]
                  print(company_income)
                  # ... rest of your code to process company_income ...
              else:
                  print(f"No matching data found for company {company_datas[index][0]}")

              # Handle potential empty results for '營業利益' and '基本每股盈餘(元)'
              result_grossloss = df_csv.query("公司代號=="+company_datas[index][0])["營業利益"]
              if not result_grossloss.empty:
                  company_grossloss = result_grossloss.values[0]
              else:
                  company_grossloss = "N/A" # Or any other placeholder for missing data

              result_eps = df_csv.query("公司代號=="+company_datas[index][0])["基本每股盈餘(元)"]
              if not result_eps.empty:
                  company_eps = result_eps.values[0]
              else:
                  company_eps = "N/A" # Or any other placeholder for missing data

              data.append([company_datas[index][0], company_datas[index][1], company_datas[index][2], company_datas[index][3], company_income, company_grossloss, company_eps, year, season])
              write_string = f"公司名稱是{company_datas[index][1]}，簡稱為{company_codes[index][2]}, 公司代號是{company_codes[index][0]}，公司類別是{company_codes[index][3]}, 此公司是屬於{company_datas[index][3]}的產業別公司，在{year}年{season}季的財務報表, 營業收入合計為{company_income}, 營業利益為{company_grossloss}, 基本每股盈餘為{company_eps}。\n"
              print(write_string)
              file.write(write_string)
          else:
              print(f"Skipping company {company_datas[index][0]} because the DataFrame is empty.")
    new_df=pd.DataFrame(data=data, columns=columns)
    new_df.to_csv('docs/income_statement.csv', index=True)
    # # 載入TXT文檔
    # input_file = "combined_data.txt"

    # # 將 TXT 另存為 PDF 文件
    # create_pdf(input_file)

In [23]:
# @title 公開資訊觀測站取得綜合損益
def write_data_tocsv_from_public(company_datas, year, season):
    with open('combined_data.txt', 'w', newline='') as file:
        for index in range(1, len(company_datas)):
            # print(company_datas[index][0], company_datas[index][1], company_datas[index][2], company_datas[index][3])
            df = get_data_df("https://mops.twse.com.tw/mops/web/ajax_t164sb04", company_datas[index][0], year, season)
            # Check if the DataFrame is empty before setting columns
            if not df.empty:
                df.columns = ['會計項目', '金額', '%', '去年同期金額', '去年同期%', 'Unnamed']
                company_income = df.query("會計項目=='營業收入合計'")["金額"].values[0]
                company_grossloss = df.query("會計項目=='營業毛利（毛損）'")["金額"].values[0]
                company_eps = df.query("會計項目=='基本每股盈餘'")["金額"].values[1]
                write_string = f"公司名稱是{company_codes[index][1]}，簡稱為{company_codes[index][2]}, 公司代號是{company_codes[index][0]}，公司類別是{company_codes[index][3]}, 此公司是屬於{company_datas[index][3]}的產業別公司，在{year}年{season}季的財務報表, 營業收入合計為{company_income}, 營業毛利（毛損）為{company_grossloss}, 基本每股盈餘為{company_eps}。\n"
                print(write_string)
                file.write(write_string)
            else:
                print(f"Skipping company {company_datas[index][0]} because the DataFrame is empty.")

In [50]:
# @title 產出上市公司綜合損益說明書
# Download the company codes from the first website
company_codes_url = "https://mopsfin.twse.com.tw/opendata/t187ap03_L.csv"
response = requests.get(company_codes_url)
company_codes_csv = response.content.decode('utf-8')

# Parse the company codes CSV data
company_codes = []
company_categories = []
reader = csv.reader(company_codes_csv.splitlines(), delimiter=',', skipinitialspace=True)
for row in reader:
    company_code = row[1]
    company_name = row[2]
    company_nickname = row[3]
    company_categories = categories.get(company_code,"Unknown")
    company_codes.append([company_code, company_name, company_nickname, company_categories])
# print(len(company_codes))

#處理Title
# titles_data = get_data_df("https://mops.twse.com.tw/mops/web/ajax_t164sb04", company_codes[1][0], '113', '01')
# company_bus = ["income", "grossloss", "eps"]
# columns = company_codes[0] + company_bus
# print(titles_data.iloc[: , 0].values.tolist())

# Download the income statement data for each company code
# data = []
# for index in range(1, len(company_codes)):
#     print(company_codes[index][0], company_codes[index][1], company_codes[index][2], company_codes[index][3])
#     df = get_data_df("https://mops.twse.com.tw/mops/web/ajax_t164sb04", company_codes[index][0], '113', '01')
#     df.columns = ['會計項目', '金額', '%', '去年同期金額', '去年同期%', 'Unnamed']
#     company_income = df.query("會計項目=='營業收入合計'")["金額"].values[0]
#     company_grossloss = df.query("會計項目=='營業毛利（毛損）'")["金額"].values[0]
#     company_eps = df.query("會計項目=='基本每股盈餘'")["金額"].values[1]
#     print(company_income, company_grossloss, company_eps)
#     data.append([company_codes[index][0], company_codes[index][1], company_codes[index][2], company_codes[index][3], company_income, company_grossloss, company_eps])

# new_df=pd.DataFrame(data=data, columns=columns)
# new_df.to_csv('income_statement.csv', index=False)


# # Write the combined data to a CSV file
# write_data_tocsv_from_public(company_codes, '113', '01')
# write_data_totxt_from_opendata(company_codes,  '112', '04')
write_data_tocsv_from_opendata(company_codes,  '112', '04')

1101
25544599.0
1102
16499750.0
1103
699142.0
1104
1908572.0
1108
1192968.0
1109
1097070.0
1110
576896.0
1201
5334450.0
1203
1557822.0
1210
25249608.0
1213
33348.0
1215
6912107.0
1216
158756079.0
1217
1159178.0
1218
2755715.0
1219
3564395.0
1220
762305.0
1225
3343898.0
1227
6795849.0
1229
3050500.0
1231
2905460.0
1232
5626346.0
1233
572238.0
1234
2639399.0
1235
17224.0
1236
573602.0
1256
869956.0
1301
48108493.0
1303
58629908.0
1304
12024428.0
1305
2904775.0
1307
2604110.0
1308
1561353.0
1309
3790749.0
1310
2226204.0
1312
3255223.0
1313
17825490.0
1314
8191861.0
1315
455502.0
1316
228932.0
1319
6174172.0
1321
1112113.0
1323
764248.0
1324
199156.0
1325
96377.0
1326
83941693.0
1337
258235.0
1338
1157835.0
1339
512319.0
1340
76198.0
1341
605623.0
1342
613001.0
1402
65409190.0
1409
10883283.0
1410
54735.0
1413
623372.0
1414
140068.0
1416
79108.0
1417
237813.0
1418
3862.0
1419
861625.0
1423
40188.0
1432
326356.0
1434
7169831.0
1435
2650.0
1436
20084.0
1437
177252.0
1438
0.0
1439
33150.0
144